In [19]:
import numpy as np
import pandas as pd
from scipy import linalg
from scipy import stats

np.set_printoptions(suppress=True)
np.seterr(divide='raise', invalid='raise', over='warn', under='ignore')

{'divide': 'raise', 'invalid': 'raise', 'over': 'warn', 'under': 'ignore'}

In [14]:
BETA_0 = -2
BETA_1 = np.log(2)

In [158]:
def generate_data(b, n):
    x = stats.norm.rvs(loc=2, scale=1, size=n)    
    mu = np.exp(x*BETA_1 + BETA_0)
    theta = stats.gamma.rvs(b, scale=1/b, size=n)
    return x, stats.poisson.rvs(theta*mu, size=n)

def estimate_by_poisson_model(x, y):
    def score(beta):
        beta_0 = beta[0]
        beta_1 = beta[1]
        return np.array([
            np.sum(y - np.exp(beta_0 + beta_1*x)),
            np.sum(x*y - x*np.exp(beta_0 + beta_1*x)),
        ])
    
    beta_hat = optimize.root(score, np.array([-2, np.log(2)]))['x']        
    mu_hat = np.exp(beta_hat[0] + beta_hat[1]*x)
    
    fisher_determinant = np.sum(mu_hat)*np.sum(x*x*mu_hat) - np.square(np.sum(x*mu_hat))
    beta_hat_variance = np.array([
        [np.sum(x*x*mu_hat), -np.sum(x*mu_hat)],
        [-np.sum(x*mu_hat), np.sum(mu_hat)]
    ])/fisher_determinant
    
    return beta_hat, beta_hat_variance

def estimate_by_quasi_likelihood(x, y):
    beta_hat, beta_hat_variance = estimate_by_poisson_model(x, y)
    
    mu_hat = np.exp(beta_hat[0] + beta_hat[1]*x)
    if np.any(mu_hat == 0):
        print(beta_hat)
        print(x)
        print(y)
    alpha_hat = np.sum(np.square(y - mu_hat)/mu_hat)/(len(y) - 2)
    
    return beta_hat, alpha_hat*beta_hat_variance    

def estimate_by_sandwich(x, y):
    beta_hat, beta_hat_variance = estimate_by_poisson_model(x, y)
    
    mu_hat = np.exp(beta_hat[0] + beta_hat[1]*x)
    #residuals = np.expand_dims((y - mu_hat)/mu_hat, -1)
    #residuals = residuals.dot(residuals.T)
    #d_transpose = np.array([mu_hat, x*mu_hat])     
    #b_hat = d_transpose.dot(residuals).dot(d_transpose.T)
    b_hat = np.sum(
        np.square(y - mu_hat)[:,np.newaxis,np.newaxis]*np.transpose(np.array([
            [np.ones_like(y), x],
            [x, x*x],
        ]), axes=[2,1,0]),
        axis=0)
    
    return beta_hat, beta_hat_variance.dot(b_hat).dot(beta_hat_variance.T)

def is_covered_by_confidence_interval(estimates, actual, variances, level=0.95):
    return np.abs(estimates - actual) <= stats.norm.isf((1 - level)/2)*np.sqrt(np.diag(variances))

In [159]:
experiments = pd.DataFrame(index=pd.MultiIndex.from_product([
    [0.2, 1, 10, 1000],
    [10, 20, 50, 100, 250, 1000],
    #[0.2], [10],
], names=['b', 'n']))

In [160]:
import logging
from scipy import optimize

np.random.seed(2018)
for b, n in experiments.index:
    print('b={}, n={}'.format(b, n))
    poisson_estimates = []
    poisson_variances = []
    
    quasi_likelihood_estimates = []
    quasi_likelihood_variances = []
    
    sandwich_estimates = []
    sandwich_variances = []
    
    for i in range(10000):
        x, y = generate_data(b, n)
        while np.sum(y > 0) < 2:
            #logging.getLogger().warning('Regenerating data.')
            x, y = generate_data(b, n)

        poisson_estimate, poisson_variance = estimate_by_poisson_model(x, y)            
        quasi_likelihood_estimate, quasi_likelihood_variance = estimate_by_quasi_likelihood(x, y)
        sandwich_estimate, sandwich_variance = estimate_by_sandwich(x, y)
                    
        poisson_estimates.append(poisson_estimate)
        poisson_variances.append(poisson_variance)        
        
        quasi_likelihood_estimates.append(quasi_likelihood_estimate)
        quasi_likelihood_variances.append(quasi_likelihood_variance)
        
        sandwich_estimates.append(sandwich_estimate)
        sandwich_variances.append(sandwich_variance)
        
    poisson_estimates = np.array(poisson_estimates)
    poisson_variances = np.array(poisson_variances)
    
    quasi_likelihood_estimates = np.array(quasi_likelihood_estimates)
    quasi_likelihood_variances = np.array(quasi_likelihood_variances)
    
    sandwich_estimates = np.array(sandwich_estimates)
    sandwich_variances = np.array(sandwich_variances)
    
    is_covered_by_confidence_interval_vectorized = (
        np.vectorize(
            lambda e, v: is_covered_by_confidence_interval(
                e, np.array([BETA_0, BETA_1]), v),
            otypes=[np.bool],
            signature='(i),(i,i)->(i)'))
    
    poisson_coverage = np.sum(is_covered_by_confidence_interval_vectorized(
        poisson_estimates, poisson_variances), axis=0)/len(poisson_estimates)
    
    quasi_likelihood_coverage = np.sum(is_covered_by_confidence_interval_vectorized(
        quasi_likelihood_estimates, quasi_likelihood_variances), axis=0)/len(quasi_likelihood_estimates)
    
    sandwich_coverage = np.sum(is_covered_by_confidence_interval_vectorized(
        sandwich_estimates, sandwich_variances), axis=0)/len(sandwich_estimates)
    
    print(poisson_coverage)
    print(quasi_likelihood_coverage)
    print(sandwich_coverage)
    
    #print(np.sum(np.abs(estimates[:,0] - BETA_0) <= np.sqrt(variances[:,0])*stats.norm.isf((1 - 0.95)/2)))
    #print(np.sum(np.abs(estimates[:,1] - BETA_1) <= np.sqrt(variances[:,1])*stats.norm.isf((1 - 0.95)/2)))    
    #print(np.mean(estimates, axis=0))

b=0.2, n=10
[0.7795 0.7649]
[0.8933 0.894 ]
[0.7957 0.7287]
b=0.2, n=20
[0.7451 0.6921]
[0.9203 0.8959]
[0.8294 0.7384]
b=0.2, n=50
[0.6952 0.6257]
[0.9324 0.8923]
[0.8686 0.8012]
b=0.2, n=100
[0.6552 0.5728]
[0.928  0.8768]
[0.8804 0.8333]
b=0.2, n=250
[0.6291 0.5449]
[0.927  0.8668]
[0.9035 0.8756]
b=0.2, n=1000
[0.5978 0.5124]
[0.9146 0.8468]
[0.9311 0.9159]
b=1.0, n=10
[0.913  0.9054]
[0.9065 0.9052]
[0.8441 0.8022]
b=1.0, n=20
[0.8879 0.8582]
[0.9288 0.9113]
[0.8798 0.8268]
b=1.0, n=50
[0.8696 0.8285]
[0.9359 0.9104]
[0.9016 0.8718]
b=1.0, n=100
[0.8523 0.8095]
[0.9322 0.8982]
[0.9127 0.8909]
b=1.0, n=250
[0.8453 0.7938]
[0.9358 0.9   ]
[0.933 0.918]
b=1.0, n=1000
[0.8339 0.7847]
[0.9262 0.8891]
[0.9434 0.9379]
b=10.0, n=10
[0.9552 0.9609]
[0.9087 0.9124]
[0.8588 0.8337]
b=10.0, n=20
[0.9487 0.9462]
[0.9304 0.9289]
[0.9021 0.875 ]
b=10.0, n=50
[0.9348 0.9346]
[0.9359 0.9337]
[0.9154 0.9008]
b=10.0, n=100
[0.9385 0.9311]
[0.9426 0.9357]
[0.929  0.9174]
b=10.0, n=250
[0.937  0.9336]